# Continuous-Time Dynamic Network Embeddings

This is the demo of [Continuous-Time Dynamic Network Embeddings](http://ryanrossi.com/pubs/nguyen-et-al-WWW18-BigNet.pdf) implemented in Stellargraph GraphML library. This demo is to show in simple steps how time respecting walks can be done on a temporal graph and how they can be used to get network embeddings. 

We compare the embeddings learnt from temporal walks with non-temporal walks in this demo. 

## Data set

We use the **Social Spammers in Evolving Multi-Relational Social Network Dataset** to demonstrate the continuous-time dynamic network embeddings method. This dataset was released by Shobeir Fakhraei with permission from if(we) Inc. as supplementary material of the paper: [Collective Spammer Detection in Evolving Multi-Relational Social Networks](http://www.cs.umd.edu/~shobeir/papers/fakhraei_kdd_2015.pdf). 
The original anonymized dataset was collected from the Tagged.com social network website.
It contains 5.6 million users and 858 million links between them.
Each user has 4 features and is manually labeled as "spammer" or "not spammer".
Each link represents an action between two users and includes a timestamp and a type.
The network contains 7 anonymized types of links. The original task on the dataset is to identify (i.e., classify) the spammer users based on their relational and non-relational features.
https://linqs-data.soe.ucsc.edu/public/social_spammer/

<a name="refs"></a>
**References**

[1] Continuous-Time Dynamic Network Embeddings. Giang Hoang Nguyen, John Boaz Lee, Ryan A. Rossi, Nesreen K. Ahmed. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2016. ([link](http://ryanrossi.com/pubs/nguyen-et-al-WWW18-BigNet.pdf))

[2] Collective Spammer Detection in Evolving Multi-Relational Social Networks.Fakhraei, Shobeir and Foulds, James and Shashanka, Madhusudana and Getoor, Lise. Proceedings of the 21th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining. KDD 2015. ([link](http://www.cs.umd.edu/~shobeir/papers/fakhraei_kdd_2015.pdf))


### Details of the method:
The overall approach of temporal random walks is  quite simple. 

Starting from a node randomly pick each the next node forward in time. 

**Choices made in the paper:**
 * Initial temporal edge selection:  *where to start the walk?*
    * Unbiased:
        * Uniformly at random select a starting edge.
    * Biased:
        * Exponential
        * Linear

 * Temporal random walks: *how to select the next edge?* 
    * Unbiased:
       
       Uniformly at random select the next edge that has a timestamp greater than the current timestamp and starts at the destination node of the current edge.
    * Biased: 
        * Exponential 
        * Linear

* Arbitrary length of walks:
    
    Unlike timeless random walks, there may not be enough options to perform a time-based random walk of a certain fixed length L. A minimum value \omega is set for a minimum viable walk. All walks of length greater than and equal to omega and less than and equal to L are considered as valid walks to extract the (target, context ) pairs.



**Stellargraph strategies:**
* Initial temporal edge selection:  *where to start the walk?* 
    * Slightly departing from the paper’s idea, given a set of head nodes, uniformly at random select a starting edge.
        
* Temporal random walks: *how to select the next edge?* 
    * Uniformly at random
    * Exponentially decaying in time
* Bidirectional walks: the walker can perform forward or bidirectional *time respecting* random walks continuting from the latest or both outer end nodes of the random walk 

In [1]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import random
from numpy.random import choice
import math

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

#### Spammers dataset
The spammers dataset can be downloaded from here: https://linqs-data.soe.ucsc.edu/public/social_spammer/. It has two components: user data and the relationships data. For efficiency, in this demo, we utilize the userID and label (spammers = 1, non-spammers = 0) from the User data. From the relationships data, we use only day 0 and relation 1 as the extant dataset is huge.

You can update `data_path` to the location of your downloaded dataset.

In [2]:
data_path = "~/data/social-spammer/" 

**User data**

In [3]:
df_users = pd.read_csv(data_path + 'usersdata.csv', sep='\t', header=None)
df_users.columns = ['userID', 'gender', 'timePassedValidation','ageGroup', 'label']
df_users['userID'] = df_users['userID'].astype('str')
df_users.head()

,userID,gender,timePassedValidation,ageGroup,label
0,1,M,0.9000,30,0
1,2,F,1.0000,20,0
2,3,M,0.1375,30,0
3,4,M,0.3875,20,0
4,5,M,0.0125,20,0


**Relationships**
Loading the relationships between the users (emails from source to target). For this demo using only 1 day (day = 0) of data and one relationship type (relations = 1).

In [ ]:
df_relations = pd.read_csv(data_path + 'relations.csv', sep ='\t', header=None)
df_relations.columns = ['day', 'time_ms', 'source', 'target', 'relations']
df_relations_day0 =  df_relations[df_relations['day'] == 0]
df_relations_day0.shape

In [ ]:
len(df_relations_day0.time_ms.unique())

In [ ]:
len(df_relations_day0.source.unique())

In [ ]:
len(df_relations_day0.target.unique())

In [ ]:
df_relations_day0.relations.unique()

In [ ]:
df_relations_day0_rel1 = df_relations_day0[df_relations_day0['relations'] == 1]
df_relations_day0_rel1.shape

In [ ]:
df_relations_day0_rel1['source'] = df_relations_day0_rel1['source'].astype('str')
df_relations_day0_rel1['target'] = df_relations_day0_rel1['target'].astype('str')

In [ ]:
# create lit of unique users from source and target
u1 = df_relations_day0_rel1.iloc[:,2]
u2 = df_relations_day0_rel1.iloc[:,3]
users_day_0 = pd.concat([u1, u2], ignore_index=True)
users_day_0 = pd.Series.unique(users_day_0)
print("In the network there are {} users out of total {} users.".format(len(users_day_0), len(df_users)))

##### Distribution of spammers and non-spammers labels

In [ ]:
spammer_dist = df_users[df_users['userID'].isin(users_day_0)].label.value_counts()
print("The distribution of spammers in our network is\n {}".format(spammer_dist))

***Only 7.7 % spammers.***

In [ ]:
df_relations_day0_rel1.head()

In [ ]:
GDay0= nx.from_pandas_edgelist(df_relations_day0_rel1, 'source', 'target', edge_attr=['time_ms'], 
                                 create_using=nx.MultiDiGraph())
print(nx.info(GDay0))

In [ ]:
node_attr = df_users[df_users['userID'].isin(users_day_0)]
values = { str(row.tolist()[0]): row.tolist()[-1] for _, row in node_attr.iterrows() }
nx.set_node_attributes(GDay0, values, 'label')

In [ ]:
degrees = [(node,val) for (node, val) in GDay0.degree()]
df = pd.DataFrame(degrees, columns=['userID', 'degree'])
df_degrees_labels = pd.merge(df, df_users[['userID', 'label']], on='userID')
df_degrees_labels.label.value_counts()

In [ ]:
df_degrees_labels.groupby(['label']).mean()

In [ ]:
df_degrees_labels[df_degrees_labels['label']==0].hist(column = ['degree'], log = True)

In [ ]:
df_degrees_labels[df_degrees_labels['label']==1].hist(column = ['degree'],log = True)

## Random walks

In [ ]:
from stellargraph import StellarGraph, StellarDiGraph
from stellargraph.data import BiasedRandomWalk
from stellargraph.data import TemporalUniformRandomWalk
from stellargraph.data import TemporalBiasedRandomWalk

#### Create a Stellargraph object

In [ ]:
edge_weight_label = "time_ms"
g = StellarGraph(GDay0, edge_weight_label=edge_weight_label)
print(g.info())

**Set parameters for the walks**

In [ ]:
nodes = list(GDay0.nodes)
n = 5
length = 20
bidirectional = True

#### Biased Random Walks

In [ ]:
rw = BiasedRandomWalk(g)

walks = rw.run(
    nodes=nodes, # root nodes
    length=length,  # maximum length of a random walk
    n=n,        # number of random walks per root node 
    p=0.5,       # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0        # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

#### Temporal Random Walks

In [ ]:
trw = TemporalRandomWalk(g)

In [ ]:
temporal_random_walks = trw.run(
    nodes=nodes, # root nodes
    length=length,  # maximum length of a random walk
    n=n,
    bidirectional=bidirectional
    step_type="uniform"
)
print("Number of random walks: {}".format(len(temporal_random_walks)))

In [ ]:
temporal_biased_random_walks = trw.run(
    nodes=nodes, # root nodes
    length=length,  # maximum length of a random walk
    n=n,
    bidirectional=bidirectional
    step_type="exponential"
)
print("Number of random walks: {}".format(len(temporal_biased_random_walks)))

### Representation Learning using Word2Vec

We use the Word2Vec [[2]](#refs) implementation in the free Python library gensim [[3]](#refs), to learn representations for each node in the graph.

We set the dimensionality of the learned embedding vectors to 128 as in [[1]](#refs).

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=1)

In [ ]:
model_temporal_uniform = Word2Vec(temporal_random_walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=1)

In [ ]:
model_temporal_biased = Word2Vec(temporal_biased_random_walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=1)

### Downstream task

The node embeddings calculated using the biased (non-temporal) and temporal walks can be used as node feature vectors in a downstream task such as node classification. 

In this example, we will use the  node embeddings to train a simple Logistic Regression classifier to predict spammers and non-spammers.

**Biased Random walks**

In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index2word  # list of node IDs
node_embeddings = model.wv.vectors  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = [ GDay0.node[node_id]['label'] for node_id in node_ids ]

In [ ]:
# X will hold the 50 input features (node embeddings)
X = node_embeddings  
# y holds the corresponding target values
y = np.array(node_targets)

**Data Splitting**

We split the data into train and test sets. 

We use 5% of the data for training and the remaining 95% for testing as a hold out test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.05, test_size=None, stratify=y)

In [ ]:
import collections
print(collections.Counter(y))
print(collections.Counter(y_train))
print(collections.Counter(y_test))

#### Classifier Training

We train a Logistic Regression classifier on the training data. 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', 
                                     np.unique(y_train), 
                                     y_train)
train_class_weights = dict(zip(np.unique(y_train), 
                               class_weights))
train_class_weights

In [ ]:
clf = LogisticRegression(verbose=0, solver='lbfgs', multi_class="auto", class_weight=train_class_weights)
clf.fit(X_train, y_train)

Predict the hold out test set.

In [ ]:
y_pred = clf.predict(X_test)

Calculate the accuracy of the classifier on the test set.

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
roc_auc_score(y_test, y_pred)

**Predicted classes**

In [ ]:
pd.Series(y_pred).value_counts()

**True classes**

In [ ]:
pd.Series(y).value_counts()

**Uniform temporal Random walks**

In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model_temporal_uniform.wv.index2word  # list of node IDs
node_embeddings = model_temporal_uniform.wv.vectors  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = [ GDay0.node[node_id]['label'] for node_id in node_ids ]

In [ ]:
# X will hold the 50 input features (node embeddings)
X = node_embeddings  
# y holds the corresponding target values
y = np.array(node_targets)

**Data Splitting**

We split the data into train and test sets. 

We use 5% of the data for training and the remaining 95% for testing as a hold out test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.05, test_size=None, stratify=y)

#### Classifier Training

We train a Logistic Regression classifier on the training data. 

In [ ]:
clf = LogisticRegression(verbose=0, solver='lbfgs', multi_class="auto", class_weight= 'balanced')
clf.fit(X_train, y_train)

Predict the hold out test set.

In [ ]:
y_pred = clf.predict(X_test)

Calculate the accuracy of the classifier on the test set.

In [ ]:
accuracy_score(y_test, y_pred)

**Predicted classes**

In [ ]:
pd.Series(y_pred).value_counts()

**True classes**

In [ ]:
pd.Series(y).value_counts()

**Biased temporal Random walks**

In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model_temporal_biased.wv.index2word  # list of node IDs
node_embeddings = model_temporal_biased.wv.vectors  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = [ GDay0.node[node_id]['label'] for node_id in node_ids ]

In [ ]:
# X will hold the 50 input features (node embeddings)
X = node_embeddings  
# y holds the corresponding target values
y = np.array(node_targets)

**Data Splitting**

We split the data into train and test sets. 

We use 5% of the data for training and the remaining 95% for testing as a hold out test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.05, test_size=None, stratify=y)

#### Classifier Training

We train a Logistic Regression classifier on the training data. 

In [ ]:
clf = LogisticRegression(verbose=0, solver='lbfgs', multi_class="auto", class_weight='balanced')
clf.fit(X_train, y_train)

Predict the hold out test set.

In [ ]:
y_pred = clf.predict(X_test)

Calculate the accuracy of the classifier on the test set.

In [ ]:
accuracy_score(y_test, y_pred)

**Predicted classes**

In [ ]:
pd.Series(y_pred).value_counts()

**True classes**

In [ ]:
pd.Series(y).value_counts()

### Visualise Node Embeddings

We retrieve the Word2Vec node embeddings that are 128-dimensional vectors and then we project them down to 2 dimensions using PCA algorithm.

### Biased Random walks

In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index2word  # list of node IDs
node_embeddings = model.wv.vectors  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = [ GDay0.node[node_id]['label'] for node_id in node_ids ]

Transform the embeddings to 2d space for visualisation

In [ ]:
transform = PCA #TSNE

trans = transform(n_components=2)
node_embeddings_2d = trans.fit_transform(node_embeddings)

In [ ]:
# draw the embedding points, coloring them by the target label (paper subject)
alpha = 0.7
label_map = { l: i for i, l in enumerate(np.unique(node_targets)) }
node_colours = [ label_map[target] for target in node_targets ]

plt.figure(figsize=(15,15))
plt.axes().set(aspect="equal")
plt.scatter(node_embeddings_2d[:,0], 
            node_embeddings_2d[:,1], 
            c=node_colours, cmap="jet", alpha=alpha)
plt.title('{} visualization of node embeddings'.format(transform.__name__))
plt.show()

### Temporal Uniform Random walks

In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model_temporal_uniform.wv.index2word  # list of node IDs
node_embeddings = model_temporal_uniform.wv.vectors  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = [ GDay0.node[node_id]['label'] for node_id in node_ids ]

Transform the embeddings to 2d space for visualisation

In [ ]:
transform = PCA #TSNE

trans = transform(n_components=2)
node_embeddings_2d = trans.fit_transform(node_embeddings)

In [ ]:
# draw the embedding points, coloring them by the target label (paper subject)
alpha = 0.7
label_map = { l: i for i, l in enumerate(np.unique(node_targets)) }
node_colours = [ label_map[target] for target in node_targets ]

plt.figure(figsize=(15,15))
plt.axes().set(aspect="equal")
plt.scatter(node_embeddings_2d[:,0], 
            node_embeddings_2d[:,1], 
            c=node_colours, cmap="jet", alpha=alpha)
plt.title('{} visualization of node embeddings'.format(transform.__name__))
plt.show()

### Temporal Biased Random walks

In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model_temporal_biased.wv.index2word  # list of node IDs
node_embeddings = model_temporal_biased.wv.vectors  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = [ GDay0.node[node_id]['label'] for node_id in node_ids ]

Transform the embeddings to 2d space for visualisation

In [ ]:
transform = PCA #TSNE

trans = transform(n_components=2)
node_embeddings_2d = trans.fit_transform(node_embeddings)

In [ ]:
# draw the embedding points, coloring them by the target label (paper subject)
alpha = 0.7
label_map = { l: i for i, l in enumerate(np.unique(node_targets)) }
node_colours = [ label_map[target] for target in node_targets ]

plt.figure(figsize=(15,15))
plt.axes().set(aspect="equal")
plt.scatter(node_embeddings_2d[:,0], 
            node_embeddings_2d[:,1], 
            c=node_colours, cmap="jet", alpha=alpha)
plt.title('{} visualization of node embeddings'.format(transform.__name__))
plt.show()